In [4]:
import numpy as np
import os
import subprocess
# assumes lkh.exe available in same root
# http://www.akira.ruc.dk/~keld/research/LKH/lkh.exe
# The Lin-Kernighan Heuristic is not an exact solver

def squares(target):
    return [x**2 for x in range(2,int(target**0.5)+1)]

def leRU(x):
    if x > 0: return x
    return 0

def sums(square,size):
    start = max(leRU(square-size),1)
    return [(x,square-x) for x in range(start,square//2 + square%2)]

def square_sums(size):
    return [x for square in squares(2*size) for x in sums(square,size)]

def tsp_array(size):
    # hamiltonian path to TSP
    arr = np.ones((size,size),int)*2
    for x,y in square_sums(size):
        arr[y-1,x-1] = 1
    return arr

# https://github.com/perrygeo

template = """NAME: {name}
TYPE: TSP
COMMENT: {name}
DIMENSION: {n_cities}
EDGE_WEIGHT_TYPE: EXPLICIT
EDGE_WEIGHT_FORMAT: LOWER_DIAG_ROW
EDGE_WEIGHT_SECTION
{matrix_s}EOF"""

def dumps_matrix(arr, name="Problem"):
    n_cities = arr.shape[0]
    width = len(str(arr.max())) + 1
    matrix_s = ""
    for i, row in enumerate(arr.tolist()):
        matrix_s += " ".join(["{0:>{1}}".format((int(elem)), width)
                              for elem in row[:i+1]])
        matrix_s += "\n"
    return template.format(**{'name': name,
                              'n_cities': n_cities,
                              'matrix_s': matrix_s})

def _create_lkh_par(tsp_path, runs):
    par_path = tsp_path + ".par"
    out_path = tsp_path + ".out"
    par = 'PROBLEM_FILE = {}\nRUNS = {}\nTOUR_FILE = {}'.format(tsp_path, runs, out_path)
    with open(par_path, 'w') as dest:
        dest.write(par)
    return par_path, out_path

def run(size,print_=True,tsp_file='problem.tsp',runs = 1):
    with open(tsp_file, 'w') as dest:
        dest.write(dumps_matrix(tsp_array(size), name=tsp_file))
    par_path, out_path = _create_lkh_par(tsp_file, runs)
    subprocess.call(['lkh', par_path])
    with open(out_path) as solution:
        lkhout = solution.readlines()
    solvable = int(lkhout[1].strip().split(' ')[-1]) < size + 2
    if print_:
        print('This is a solvable problem is {}'.format(solvable))
        if solvable: print([int(x) for x in lkhout[6:-2:1]])
    return solvable

In [5]:
for x in range(25,200):
    assert run(x,False) == True

In [3]:
%timeit -n1 -r1 run(40000)

KeyboardInterrupt: 